In [1]:
from typing import List, Dict
from fastFM import als
from scipy.sparse import csr_matrix
import MeCab
import numpy as np
import gensim
import pandas as pd
import csv
from gensim import matutils
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tqdm import tqdm
from copy import deepcopy
from sklearn.externals import joblib
from sklearn import linear_model
import codecs
from gensim.models import word2vec
from gensim.models import KeyedVectors
from sklearn.metrics import classification_report
import math

/home/s15t283/svm_test_d/env/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## 分散表現の読み込み
- めっちゃ時間かかる
- これは一度実行した後は，なんども実行しなくてOK

In [2]:
file_w2v_hottolink = "w2v_all_vector200_win5_sgns0.vec"
model_hottolink = KeyedVectors.load_word2vec_format(file_w2v_hottolink, binary=False)
#model_hottolink = word2vec.Word2Vec.load("word2vec.model")

### 関数の定義

In [3]:
# 文章をmecabで分かちがきして、名詞・動詞・形容詞の単語一覧を返す
def wakati(text):
    tagger = MeCab.Tagger()
    tagger.parse('')
    node = tagger.parseToNode(text)
    word_list = []
    while node:
        features = node.feature.split(",")
        pos = features[0]
#        if pos in ["名詞", "動詞", "形容詞", "助動詞"]:
        lemma = node.surface
        word_list.append(lemma)
            # 標準形に変更
        node = node.next

    return word_list[1:-1]

def parse():
    lines = []
    #for line in open('lastdata1_text.txt', 'r'):
    for line in open('./sample_LDA_setdata/all_smptons_set1000_text.txt', 'r'):
        arr = line.split("\t")
        lines.append(arr)
    return lines

def vec2dense(vec, num_terms):
    return list(matutils.corpus2dense([vec], num_terms=num_terms).T[0])
#    return np.array(list(matutils.corpus2dense([vec], num_terms=num_terms).T[0]))
def scores2mean(scores: List[float]) -> float:
    return sum(scores) / len(scores)

## 分散表現の読み込み

In [9]:
wakatis = []
wakatis1 = []
wakatis_bow = []
wakatis_bow1 = []
wakatis_bow = pd.read_csv("./sample_LDA_setdata/all_smptons_set1000_tsutsuzi.csv",header = None).values.tolist()
wakatis_bow1= pd.read_csv("./sample_LDA_setdata/all_smptons_set1000_LDA.csv",header = None).values.tolist()
print(len(wakatis_bow[0]))
print(len(wakatis_bow1[0]))
print(len(wakatis_bow))
vectors = []
vectors_bow = []
lines = [t[0] for t in parse()]
for line in lines:
#    wakatis_bow.append(wakati(line))
    words = wakati(line)    
    embeddings = np.zeros(200)
    count = 0
    for line1 in words:
        if line1 in model_hottolink.wv:
            embeddings += model_hottolink[line1]
            count += 1
    if count > 0:
        embeddings /= count
    vectors.append(deepcopy(embeddings))

print(len(vectors))
    

#  素性追加  ##  

"""
f2 = open('./fit_data/text_word_count.csv', 'r')
dataReader = csv.reader(f2)
for i , row in enumerate(dataReader):
    vectors[i] = np.append(vectors[i],float(row[0]))

f = open('./fit_data/text_body_part.csv', 'r') 
dataReader = csv.reader(f)
for i , row in enumerate(dataReader):
    vectors[i] = np.append(vectors[i],float(row[0]))
"""



5
50
1701


/home/s15t283/svm_test_d/env/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


1701


"\nf2 = open('./fit_data/text_word_count.csv', 'r')\ndataReader = csv.reader(f2)\nfor i , row in enumerate(dataReader):\n    vectors[i] = np.append(vectors[i],float(row[0]))\n\nf = open('./fit_data/text_body_part.csv', 'r') \ndataReader = csv.reader(f)\nfor i , row in enumerate(dataReader):\n    vectors[i] = np.append(vectors[i],float(row[0]))\n"

## 追加素性
- 体の部位
- 病名
- 単語数

- その他考えられそうなもの
    - 文字数（文長）
    - BoW, PMI, word2vecを変えながら実験してみる
    - つつじ，ずんだ，この辺りはまた今度考える

In [19]:
"""
#  素性追加  ##
f = open('./text_body_part1.csv', 'r') 
#f = open('./fit_data/text_body_part.csv', 'r') 
dataReader = csv.reader(f)
for line , row in zip(vectors , dataReader):
    np.append(line, float(row[0]))
    # line.append(float(row[0]))
    # ↑word2vecの場合は，`vectors`がnumpyベクトルであるため，別のメソッドを使っている

f1 = open('./text_sick1.csv', 'r')
#f1 = open('./fit_data/text_sick.csv', 'r')
dataReader1 = csv.reader(f1)
for line , row in zip(vectors , dataReader1):
    np.append(line, float(row[0]))
    # line.append(float(row[0]))

f2 = open('./fit_data/text_word_count.csv', 'r')
dataReader2 = csv.reader(f2)
for line, row in zip(vectors , dataReader2):
    np.append(line, float(row[0]))
    # line.append(float(row[0]))


#  pmi  #
ar_1 = vectors
eps = 1e-8
cm = np.array(ar_1)
pmi = np.zeros_like(cm,dtype=np.float32)
nsum = np.sum(cm)
ssum = np.sum(cm,axis=0)
for i in range(cm.shape[0]):
    if i==200:
         break
    for j in range(cm.shape[1]):
         pw = np.log2(cm[i,j] * nsum / (ssum[j] * ssum[i]) + eps)         
         pmi[i,j] = max(0,pw)
vectors = pmi
"""
"""
f = open('./tsutsuzi_label_sample2.csv', 'r') 
dataReader = csv.reader(f)
for line , row in zip(wakatis_bow , dataReader):
    if str(row[0]) != "0.0":
        line.append(str(row[0]))
  
f1 = open('./tsutsuzi_label_sample2.csv', 'r')
dataReader1 = csv.reader(f1)
for line1 , row1 in zip(wakatis_bow , dataReader1):
    if str(row1[1]) != "0.0":
        line1.append(str(row1[1]))
 
f2 = open('./tsutsuzi_label_sample2.csv', 'r')
dataReader2 = csv.reader(f2)
for line2 , row2 in zip(wakatis_bow , dataReader2):
    if str(row2[2]) != "0.0":
        line2.append(str(row2[2]))

f3 = open('./tsutsuzi_label_sample2.csv', 'r')
dataReader3 = csv.reader(f3)
for line3 , row3 in zip(wakatis_bow , dataReader3):
    if str(row3[3]) != "0.0":
        line3.append(str(row3[3]))

f4 = open('./tsutsuzi_label_sample2.csv', 'r')
dataReader4 = csv.reader(f4)
for line4 , row4 in zip(wakatis_bow , dataReader4):
    if str(row4[4]) != "0.0":
        line4.append(str(row4[4]))
 
f5 = open('./tsutsuzi_label_sample2.csv', 'r')
dataReader5 = csv.reader(f5)
for line5 , row5 in zip(wakatis_bow , dataReader5):
    if str(row5[5]) != "0.0":
        line5.append(str(row5[5]))

f6 = open('./tsutsuzi_label_sample2.csv', 'r')
dataReader6 = csv.reader(f6)
for line6 , row6 in zip(wakatis_bow , dataReader6):
    if str(row6[6]) != "0.0":
        line6.append(str(row6[6]))
  
f7 = open('./tsutsuzi_label_sample2.csv', 'r')
dataReader7 = csv.reader(f7)
for line7 , row7 in zip(wakatis_bow , dataReader7):
    if str(row7[7]) != "0.0":
        line7.append(str(row7[7]))
"""


dictionary = gensim.corpora.Dictionary(wakatis_bow)
dictionary.filter_extremes(no_below=1, no_above=0.9)
print(dictionary)
vectors_bow = np.array([vec2dense(dictionary.doc2bow(wakatis_bow[i]), len(dictionary)) for i in range(len(wakatis_bow))], dtype='f8')
print(vectors_bow[1])

##  LDA  ##
combined_vectors = []
for bow, w2v, in zip(wakatis_bow1, vectors):
     combined_vectors.append(np.hstack((bow,w2v)))
print(len(combined_vectors[0]))
print(len(combined_vectors))

##  つつじ  ##
np.insert(combined_vectors,[200],vectors_bow,axis=1)
combined_vectors1 = []
for bow, w2v in zip(vectors_bow, combined_vectors):
    combined_vectors1.append(np.hstack((bow,w2v)))
print(len(combined_vectors1[0]))
print(len(combined_vectors1))

print(combined_vectors[0])
print(combined_vectors1[5])

Dictionary(60 unique tokens: ['0951Q.1xx.03n01', '0751M.1xx.03n01', '0961Q.4xx.03n01', '3261M.1xx.46n01', '0961Q.2rx.03n01']...)
[0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
250
1701
310
1701
[ 0.          0.13988405  0.          0.          0.          0.
  0.06679823  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.24481283  0.17313408  0.          0.
  0.08548806  0.          0.          0.          0.          0.
  0.3184396   0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          1.38449832 -1.91864789 -1.36573228 -0.44868183
 -0.43701794  0.75139243  0.94758765 -1.05211136 -1.12704403 -1.25714665
 -0

In [6]:
#labels = np.loadtxt('lastdata1_ans.csv',  delimiter=',', dtype='int')
labels = np.loadtxt('./sample_LDA_setdata/all_smptons_set1000_ans.csv',  delimiter=',', dtype='int')

kf = KFold(n_splits=10, random_state=0, shuffle=True)
scores_dict_format: Dict[str, List[float]] = {
    'precision': [], 'recall': [], 'f_measure': [], 'precision_weighted':[], 'recall_weighted':[], 'f1_weighted':[]}

In [7]:
results: Dict[int, Dict[str, List[float]]
              ] = {-1: deepcopy(scores_dict_format), 1: deepcopy(scores_dict_format)}
accuracies = []
for train_index, test_index in tqdm(kf.split(combined_vectors, labels)):
    train_data = [combined_vectors[idx] for idx in train_index]
    #train_data = np.array([combined_vectors[idx] for idx in train_index])
    train_labels = [labels[idx] for idx in train_index]
    #train_labels = np.array([labels[idx] for idx in train_index])
    test_data = [combined_vectors[idx] for idx in test_index]
    test_labels = [labels[idx] for idx in test_index]
                   
    #model = als.FMClassification(n_iter=1000, init_stdev=0.1, rank=2, l2_reg_w=0.1, l2_reg_V=0.5)
    #model.fit(csr_matrix(train_data), train_labels)
    
    ##  ロジスティックス回帰  ##
    #model = linear_model.LogisticRegression(random_state=0)
    #model.fit(train_data, train_labels)
    
    ##  SVM  ##
    #model = SVC(kernel='rbf', random_state=0, C=1000)#, gamma="scale")
    #model.fit(train_data, train_labels)
    
    ## Multi layer perceptron ##
    model = MLPClassifier(solver='sgd', batch_size=128)
    model.fit(train_data, train_labels)
    
    pred_labels = model.predict(test_data)
    accuracies.append(accuracy_score(test_labels, pred_labels))
    p, r, f, _ = precision_recall_fscore_support(test_labels, pred_labels)
    results[-1]['precision'].append(p[0])
    results[-1]['recall'].append(r[0])
    results[-1]['f_measure'].append(f[0])
    results[1]['precision'].append(p[1])
    results[1]['recall'].append(r[1])
    results[1]['f_measure'].append(f[1])
    
    results[-1]['precision_weighted'].append(p[0])
    results[1]['precision_weighted'].append(p[1])
    results[-1]['recall_weighted'].append(p[0])
    results[1]['recall_weighted'].append(p[1])
    results[-1]['f1_weighted'].append(p[0])
    results[1]['f1_weighted'].append(p[1])
    
    print(classification_report(test_labels,model.predict(test_data)))

#    for c , p , t in zip(test_labels , pred_labels , lines):
#        if c != p:
#            print(c,p,t)

    #df = list(d.items())
    #print (d.keys())
    
    # print('class: -1', results[-1])
print('\nclass: -1(macro_mean)',
      '\nprecision:', scores2mean(results[-1]['precision']),
      '\nprecision_weighted:', scores2mean(results[-1]['precision_weighted']),
      'recall', scores2mean(results[-1]['recall']),
      'recall_weighted:', scores2mean(results[-1]['recall_weighted']),  
      'f_measure', scores2mean(results[-1]['f_measure']),
      'f1_weighted:', scores2mean(results[-1]['f1_weighted']))
# print('class:  1', results[1])
print('\nclass:  1(macro_mean)',
      '\nprecision:', scores2mean(results[1]['precision']),
      '\nprecision_weighted:', scores2mean(results[1]['precision_weighted']),
      'recall', scores2mean(results[1]['recall']),
      'recall_weighted:', scores2mean(results[1]['recall_weighted']),
      'f_measure', scores2mean(results[1]['f_measure']),
      'f1_weighted:', scores2mean(results[1]['f1_weighted']))

#print(((scores2mean(results[-1]['precision_weighted']))+(scores2mean(results[1]['precision_weighted']))) / 2.0)
#print(((scores2mean(results[-1]['recall_weighted']))+(scores2mean(results[1]['recall_weighted']))) / 2.0)
#print(((scores2mean(results[-1]['f1_weighted']))+(scores2mean(results[1]['f1_weighted']))) / 2.0)

0it [00:00, ?it/s]/home/s15t283/svm_test_d/env/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
1it [00:04,  4.58s/it]

              precision    recall  f1-score   support

           0       0.85      0.93      0.89       113
           1       0.83      0.67      0.74        58

    accuracy                           0.84       171
   macro avg       0.84      0.80      0.81       171
weighted avg       0.84      0.84      0.84       171



/home/s15t283/svm_test_d/env/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
2it [00:09,  4.56s/it]

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       116
           1       0.75      0.67      0.71        54

    accuracy                           0.82       170
   macro avg       0.80      0.78      0.79       170
weighted avg       0.82      0.82      0.82       170



/home/s15t283/svm_test_d/env/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
3it [00:13,  4.56s/it]

...

              precision    recall  f1-score   support

           0       0.87      0.88      0.88       111
           1       0.77      0.75      0.76        59

    accuracy                           0.84       170
   macro avg       0.82      0.81      0.82       170
weighted avg       0.83      0.84      0.83       170



/home/s15t283/svm_test_d/env/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
9it [00:41,  4.55s/it]

              precision    recall  f1-score   support

           0       0.85      0.86      0.86       111
           1       0.74      0.71      0.72        59

    accuracy                           0.81       170
   macro avg       0.79      0.79      0.79       170
weighted avg       0.81      0.81      0.81       170



/home/s15t283/svm_test_d/env/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
10it [00:45,  4.56s/it]

              precision    recall  f1-score   support

           0       0.90      0.85      0.88       116
           1       0.72      0.80      0.75        54

    accuracy                           0.84       170
   macro avg       0.81      0.82      0.82       170
weighted avg       0.84      0.84      0.84       170


class: -1(macro_mean) 
precision: 0.8685719217204758 
precision_weighted: 0.8685719217204758 recall 0.8889907109048816 recall_weighted: 0.8685719217204758 f_measure 0.8782198431313836 f1_weighted: 0.8685719217204758

class:  1(macro_mean) 
precision: 0.7478457367758407 
precision_weighted: 0.7478457367758407 recall 0.7064228550389894 recall_weighted: 0.7478457367758407 f_measure 0.7246202227327534 f1_weighted: 0.7478457367758407


### svm 
#### word2vec
Fclass: -1(macro_mean) 
precision: 0.7559140299372962 recall 0.7144076721450139 f_measure 0.7323472242955001

class:  1(macro_mean) 
precision: 0.7122171063423284 recall 0.7582533810218722 f_measure 0.7324821540127525

#### word2vec+病名
class: -1(macro_mean) 
precision: 0.7558769307820346 recall 0.7144076721450139 f_measure 0.7323006004212901

class:  1(macro_mean) 
precision: 0.7123384347274013 recall 0.7586004614628685 f_measure 0.7326731272440975

#### word2vec+部位
class: -1(macro_mean) 
precision: 0.7510065186004868 recall 0.7155715164706229 f_measure 0.7305426192252337

class:  1(macro_mean) 
precision: 0.7113255195828777 recall 0.7505846411662427 f_measure 0.728373567598106

#### word2vec+文長 
class: -1(macro_mean) 
precision: 0.7549770930003593 recall 0.7144076721450139 f_measure 0.7318203752580128

class:  1(macro_mean) 
precision: 0.7119840061092282 recall 0.7570905903241977 f_measure 0.7317339870218801

#### word2vec+3素性
class: -1(macro_mean) 
precision: 0.7497172677967044 recall 0.7143950458823876 f_measure 0.7291640620550615

class:  1(macro_mean) 
precision: 0.7103821947808755 recall 0.7498081261016211 f_measure 0.7273705901392942

### MLP
#### word2vec
class: -1(macro_mean) 
precision: 0.7190290388941041 recall 0.7316442283000363 f_measure 0.7233889547336847

class:  1(macro_mean) 
precision: 0.7057989254463405 recall 0.6968113618494095 f_measure 0.6993334167722509

#### word2vec+病名
class: -1(macro_mean) 
precision: 0.7265466291598826 recall 0.7153768202716453 f_measure 0.7187895540245088

class:  1(macro_mean) 
precision: 0.7006216137788077 recall 0.7133205601221 f_measure 0.7046645183156733

#### word2vec+部位
class: -1(macro_mean) 
precision: 0.7261581534651302 recall 0.7130075463545308 f_measure 0.7171599673538495

class:  1(macro_mean) 
precision: 0.6993122187295194 recall 0.7165268780924017 f_measure 0.7051272209948094

#### word2vec+文長
class: -1(macro_mean) 
precision: 0.7390313823692203 recall 0.7199383524765134 f_measure 0.7271926104241068

class:  1(macro_mean) 
precision: 0.7080127913895308 recall 0.7282764160763009 f_measure 0.715977786588644

#### word2vec+3素性
class: -1(macro_mean) 
precision: 0.7232418720450512 recall 0.7210102333547977 f_measure 0.7199297460710744

class:  1(macro_mean) 
precision: 0.703361281312904 recall 0.7072590589736242 f_measure 0.7031391663512795

### ロジスティックス回帰
#### word2vec


#### word2vec+病名


#### word2vec+部位


#### word2vec+文長


#### word2vec+3素性
class: -1(macro_mean) 
precision: 0.7226595176162555 recall 0.6885464518934363 f_measure 0.7031535607992214

class:  1(macro_mean) 
precision: 0.6843165868658938 recall 0.7213928781468446 f_measure 0.7000704210006579

In [76]:
len(combined_vectors[0])

246

In [9]:
1 -1 それはどうかな庶民は前半は無難な質問を投げかけつつ痺れを切らしたインサイダーの誘導が始まったあたりで全力で答えを考えてなんとなく近い言葉ばかり発して行くとインサイダーが乗り出すからそこを狙ってインサイダーを誘導するのが頭いい。

1 -1 もぐもぐ症候群とは定期的にグウェンを摂取しないと震えや動悸、めまい、吐き気、手足の痺れなどの症状が出る病気。

-1 1 発熱で休んだ元気な幼児がいます(しんどい

-1 1 痛みは数日でおさまりましたが、下顎の痺れは半年経った現在でも少し残ってしまいました。

-1 1 インフルエンザとかになって有休消化したいとか思ってたけど間違ってたしんどいわ。

1 -1 ）」ってなったんだけど帰って鏡見たら目がすげー充血してるし目の下に漫画みたいな濃さのクマもできてたのでびっくらこいた（でも元気）

1 -1 めまいが…

1 -1 思えば、先週の木曜日から次男発熱。

1 -1 っていう声とニヤニヤと動悸が止まらないんですけどどうしたらいいですか？

1 -1 自分キモすぎて吐き気するわwなんでこんな性格なんだろね

1 -1 カンタくんに影響されて始めたランニング🏃♂️🏃♀️週4〜5回走るのを2週間続けたら、息切れからの回復速度が速くなった！

-1 1 夏のめまいfeat.

-1 1 頭痛薬キメてたせいで献血できなかった………元気です！

-1 1 昨日から少し頭痛していて、薬飲んでもあんま効かねーし、更に今朝起きたら動くたびに痛みがずきずきくるもんだからなんぞ？

1 -1 雨からの急な晴れの日は動悸がしてホントしんどい…心を無にして電車に乗る夏がもうすぐやってくる！

1 -1 晴れてるのに関節痛がひどい.

1 -1 手の甲には食器用洗剤に負けて発疹が出来ちょったけど、入院してからどっちも綺麗に治った。

-1 1 動悸というか、急に心臓がバクバクするのは不整脈？

-1 1 うわ～立ったら、少しめまいっぽくなった…そして、飛蚊症も、ちらっとした(-""-;)最近、ゲームのし過ぎかな…眼精疲労だなぁ

-1 1 これっていう事なんだとして関節痛が過ぎている気持ちがするんだけど 

-1 1 頭痛いい眠いしだるいし足筋肉痛っぽいし

1 -1 その後に突発性発疹が出るかもねとも言われた。

-1 1 っと思ってたけど今朝すっかり熱は下がりました。

-1 1 発疹は気になるけど、発熱もなく、ミルクもしっかり飲めてるなら様子見でいいんじゃないか」と、安心していいんだか、じゃあこの発疹はなんなんだと心配は消えず。

1 -1 7日から5連勤の予定だったけど5日間発熱し倒してて土曜日は39度、日曜日は吐き気酷くて休日診療所で胃腸炎と言われ1日分の薬を貰い、どーにか10日に仕事復帰したのにその夜に食べた何かしらで全身蕁麻疹でて、次の日の朝に赤い発疹に変わり、休憩時間に行った病院で

1 -1 吐き気やば

1 -1 )皆もこんな動悸するんだ？

-1 1 まぁ一週間ほどこのまま使ってみますかー時々触ってどれくらい熱くなるのか見とこもともと結構発熱するけど

-1 1 流石に39度近くまで行った発熱は一晩じゃ治らんわな。

-1 1 昨日紹介状もらって病院行ったけど手足口病じゃなかった笑2度目の突発性発疹！

1 -1 ●娘の症状足の裏から、ふともも下にかけて赤い発疹がたくさん出る遅れて、手指、腕にかけて発疹翌日手足の指先付近に水疱がでる

-1 1 風邪キメたんですけど発熱と咳に効く総合風邪薬買った直後本当にやばいのは鼻の方だって事に気づいてしまったので鼻に効くやつ買えばよかったわ〜〜〜って思いました 

1 -1 そう言えば昨日は卵焼きとビールしか胃に入れてないな、朝起きた時吐き気やばくてびっくりした

1 -1 うえー頭痛

-1 1 まじで関節痛やばい仮病で仕事休んで平日休みたい

-1 1 昨日まで40度近くあった熱も今日は平熱で安心！

1 -1 知らない人がぶつかってくるだけで吐き気する

-1 1 自分の画力が低すぎて地の底を這ってるレベルなせいで動悸してきた

-1 1 発熱 

-1 1 ただし頭の後ろにゴムを回すタイプだと髪を巻き込むのが気になるし、長さをうまく調整しないと締め付け感で軽い頭痛っぽくなる。

-1 1 昔からそうだけど、自分の予定を急に誰かに乱されるのが実際に吐き気を催すくらいに嫌いっぽいなぁどうにかならんかなコレ

1 -1 朝起きたら雪山でゴーグルしなかった時みたいに目が疲れて充血してるんですけど昨日の羽生結弦さんがあまりにも眩しすぎたからかな…👼🏻🌤

-1 1 腰は3回目だけど、入院中とか全く痛くなくて、今回痺れまくってるの何これ…



SyntaxError: invalid character in identifier (<ipython-input-9-42fc1ebba3b5>, line 1)

In [ ]:
class: -1(macro_mean) 
precision: 0.8597758500776866 
precision_weighted: 0.8597758500776866 recall 0.8883631038211288 recall_weighted: 0.8597758500776866 f_measure 0.8734145898578933 f1_weighted: 0.8597758500776866

class:  1(macro_mean) 
precision: 0.7339829286358177 
precision_weighted: 0.7339829286358177 recall 0.6740374147239484 recall_weighted: 0.7339829286358177 f_measure 0.6994706189692754 f1_weighted: 0.7339829286358177